# Lyrics Using Genius API

In [1]:
pip install lyricsgenius

Note: you may need to restart the kernel to use updated packages.


In [31]:
#import libraries
import pandas as pd
import lyricsgenius as genius #used to interface with Genius API
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import nltk
nltk.download('omw-1.4')

In [32]:
#token provided by Genius API
%store -r client_access_token

no stored variable or alias client_access_token


In [33]:
client_access_token = 'qqoXje06ERBkSJz9qtBBl1HzJyLoxuC51bU2bftPjMgS1XnkUeyVQ5_9ZNjvL6g7'

In [34]:
#initiate Genius
genius = genius.Genius(client_access_token)
genius.verbose = False #turn off status messages

### Functions

In [35]:
def get_lyrics(track,artist):
    '''
    function returns song's lyrics
    parameters:
        track-->str
        artist-->str
    '''
    track = re.sub(' - .+','',track) #remove text after '-'
    track = re.sub(' \(.*\)','',track) #remove text within parentheses
    track = re.sub(' \[.*\]','',track) #remove text within brackets
    
    try:
        return genius.search_song(track,artist).lyrics
    except:
        print(track + ' by ' + artist + ' is not available')

In [36]:
def get_df_songs(track_list,artist_list):
    '''
    function obtains lyrics and returns dataframe with columns for track, artist, lyrics
    parameters:
        track_list-->list of str 
        artist_list-->list of str
    '''
    lyrics_list = [get_lyrics(track_list[x],artist_list[x]) for x in range(len(track_list))] #get lyrics for each song
    
    return pd.DataFrame(data={'track':track_list,'artist':artist_list,'lyrics':lyrics_list})

In [37]:
def clean_lyrics(df,col,new_col):
    '''
    function returns dataframe with new column of cleaned text (song lyrics)
    parameters:
        df-->pandas dataframe
        col-->column to clean (str)
        new_col-->name of column with cleaned text (str)
    '''
    df[new_col] = df[col].str.lower() #make all text lowercase
    df[new_col] = df[new_col].str.replace(r'\n',' ') #replace '\n' character with space
    df[new_col] = df[new_col].str.replace(r'\[[^\[\]]*]','') #remove brackets and inside text
    df[new_col] = df[new_col].str.replace(r"\'\w*",'').str.replace(r'[^\w\d\s]+','') #remove extra characters
    df[new_col] = df[new_col].str.strip() #remove extra whitespace
    
    return df

In [38]:
def normalize_lyrics(df,col):
    '''
    function returns dataframe with column as list of words
        tokenizes, removes stopwords from, and lemmatizes lyrics
    parameters:
        df-->pandas dataframe
        col-->column to normalize
    '''

    df[col] = df[col].str.split() #tokenize lyrics
    
    stop_words = set(stopwords.words('english'))
    df[col] = df[col].apply(lambda row: [w for w in row if w not in stop_words]) #remove stopwords

    lemmatizer = WordNetLemmatizer()
    
    def lemmatize_text(text):
        '''
        function returns lemmatized text
        parameters:
            text-->str
        '''
        return [lemmatizer.lemmatize(w) for w in text]
    
    df[col] = df[col].apply(lemmatize_text) #lemmatize words

    return df

In [39]:
import re
def segmenting(s):
    s = s.replace('\n',' ')
    structures = re.findall(r'\[.*?\]', s)
    iter = re.finditer(r'\[.*?\]', s)
    indices = [m.start(0) for m in iter]
    splits = []
    for i in range(len(indices)):
        if i != len(indices)-1:
            splits.append((structures[i], s[indices[i]+len(structures[i]): indices[i+1]]))
        else:
            splits.append((structures[i], s[indices[i]+len(structures[i]):]))
    return splits

### Import Songs to Analyze

Read in the resulting dataframes from the spotify_analysis notebook, which were created as follows:

 - Started with the top five tracks for each of country, R&B/hip-hop, and rock/alternative as of the week of May 15, 2021, based on Billboard Top 100 charts (referred to as the "seed tracks")
 - Used Spotify's recommender algorithm to find the most similar songs to the seed tracks (returns a maximum of 100 songs per search)
 - Ranked the most similar songs by audio features using Euclidean distance
 - Fed the top ranking songs through Spotify's recommender algorithm until there were at least 1,000 songs per genre

In [12]:
#rock/alternative
df_rock = pd.read_csv('../Data/rock_songs_30k.csv')
df_rock.drop(columns='Unnamed: 0',inplace=True)

### Obtain Lyrics from Genius API

Pull lyrics from the Genius API with the lyricsgenius wrapper and put into dataframes.

In [14]:
#rock
#df_rock_lyrics = get_df_songs(df_rock['track'],df_rock['artist'])

Drifter by Hippie Sabotage is not available
Smooth Criminal by Annapantsu is not available
Bert på Jamaica by Bert & Herman Hunters is not available
Happy 'Cause I'm Goin' Home by Charles Earland is not available
Cantaloupe Island by Brian Bromberg is not available
Mumien im Autokino by Welle: Erdball is not available
All I Want by Staind is not available
Watermelon Man by Bernie Worrell is not available
Hush by VRSTY is not available
La Luh by Choomba is not available
Ethereal by Memtrix is not available
Sweet Child O' Mine feat. Gilby Clarke & Tracii Guns by Love/Hate is not available
Werewolves of London by Warren Zevon is not available
Blues Force by Fourplay is not available
Goblins in Deadwood by SpellBlast is not available
Seems to Me by Jaga Jazzist is not available
Balkan Express by Gramatik is not available
I'm Ready by Commodores is not available
It's Over by Silverstein is not available
Little Sunflower by Freddie Hubbard is not available
Rodeo Cowboy by Garry Lee and Showd

Play by Dave Grohl is not available
Psychotubes by Moon Hooch is not available
Arpeggios from Hell by Yngwie Malmsteen is not available
Hallogallo by NEU! is not available
Maiden Voyage by Herbie Hancock is not available
Apaixonei De Vez by Guga Nandes is not available
Mazurka by Mink DeVille is not available
Ronka by Perverts (swe) is not available
Chateau by Rob Dougan is not available
Gritz by Tropidelic is not available
Queen St. Gang by Arzachel is not available
Spooky Girls Are Ruining My Life! by Gxrefck is not available
The Macrocosm by Rings of Saturn is not available
Flacko by Joel Fletcher is not available
Annul by Ed Harrison is not available
I Teleported to Acapulco by Øresund Space Collective is not available
Motorcycle by Blackwater Holylight is not available
Deviant by Dick Dynamite And The Doppelgangers is not available
Chaos in the Jungle by Pim Stones is not available
Glenn by Slint is not available
Messaien's Gumbo by John Patitucci is not available
Världen är din b

Lesson One by Russ Conway is not available
Tsitsushka by Thy Catafalque is not available
Go the Distance by Peyton Parrish is not available
The Owl House Opening Cover by Frostfm is not available
The Rat Catcher by Wucan is not available
Big Country by Béla Fleck and the Flecktones is not available
The 50 Stroke by Adam McClure is not available
Cornelius by The Bloody Beetroots is not available
Tamas by My Sleeping Karma is not available
Never Tear Us Apart by Dylan LeBlanc is not available
The Way Up: Opening & Part One by Pat Metheny Group is not available
Kuu Erkylän Yllä by Havukruunu is not available
Motorcycle Emptiness by Manic Street Preachers is not available
Byrralku Dhangudha by ICEHOUSE is not available
Says by Nils Frahm is not available
Panacea by Disasterpeace is not available
Pneumonia Hawk by Cancer Bats is not available
Black Sheep of the Family by Ritchie Blackmore's Rainbow is not available
The Epilogue by ✝✝✝ (Crosses) is not available
Disco Science by Mirwais is n

Infinite Sequential Visions of a Sphere of Hate by And Hell Followed With is not available
Night Rhythms by Lee Ritenour is not available
Hit Megamix by Saragossa Band is not available
Phantom Pt. II by Justice is not available
Ninguém Vai Dormir by MC G15 is not available
Ta Faltando Um by Wealstarcks is not available
Mona Lisa Overdrive by Juno Reactor is not available
L'alba dei morti viventi by Goblin is not available
Spend The Night Together by Garry Lee and Showdown is not available
Brindo by Mario Bautista is not available
Original MTK #3 by MTK is not available
Starlight Glitz by Pete Rock is not available
Speaking Gently by BADBADNOTGOOD is not available
Venture by Jaron is not available
Baba Drame by Bill Frisell is not available
Storm Returns by Prefuse 73 is not available
Crystal Silence by Chick Corea is not available
House Of Jealous Lovers by The Rapture is not available
Sheez Music by Dumpstaphunk is not available
Vordt of the Boreal Valley by Motoi Sakuraba is not avai

Hell March by Frank Klepacki is not available
Mozart by Mat Zo is not available
Run Fay Run by Isaac Hayes is not available
If I Had $1,000,000 by Barenaked Ladies is not available
Hell March by Frank Klepacki is not available
Riding by Daniel Norgren is not available
Gonna Be by Rosie Lowe is not available
Futurmastic by Dombresky is not available
Noblesse Oblige Richesse Oblige by Wiegedood is not available
Welcome to Lunar Industries by Clint Mansell is not available
A Curved Blade by Atrium Carceri is not available
Home by Medasin is not available
Under The Thunder by Skegss is not available
School Days by Stanley Clarke is not available
Don't Worry 'Bout What I Do by Delvon Lamarr Organ Trio is not available
Blacktooth Grin by Whiskeydick is not available
TAKE ME by CASIOPEA is not available
Chewier by Ozric Tentacles is not available
Flores Amarillas by Floricienta is not available
Sunday Sermon by Booker T. & the M.G.'s is not available
Delusional by Blackwater Holylight is not 

...

It's a Mystery by Toyah is not available
Doubt by Shallou is not available
Oversentimental by The Wldlfe is not available
The Journey by Sol Rising is not available
Zombie Walk by Desiigner is not available
I Won't Let The Sun Go Down On Me by Nik Kershaw is not available
you don't listen to me anyway by PULSE is not available
Delta Dawn by Tanya Tucker is not available
Triphekta by Downlink is not available
American Soldier by Toby Keith is not available
I'm A Swing It by House Of Pain is not available
Get Over It by Lower Than Atlantis is not available
On Your Mind by Noah Floersch is not available
Back In The Mud by Bubba Sparxxx is not available
Jaded by Takida is not available
Galveston by Glen Campbell is not available
George Strait Songs by Logan Mize is not available
Snow by Angus & Julia Stone is not available
A Higher Level of Hate by Fit For An Autopsy is not available
Alla vill till himmelen men ingen vill dö by Mimikry is not available
Meeting In My Bed Room by Silk is not

Malibu 1992 by COIN is not available
Open Your Eyes by Alter Bridge is not available
Bloodline by Aephanemer is not available
Everything Zen by Bush is not available
Strangler by Orbit Culture is not available
Epiphany by Mammoth WVH is not available
Bye Bye Love by The Everly Brothers is not available
There's A Kind Of Hush by Herman's Hermits is not available
What Katie Did by The Libertines is not available
Ocean Blue by Cut Copy is not available
Island by Aer is not available
Fire by Mother's Finest is not available
Absolute Beginners by David Bowie is not available
Lay It Down by Peter Bjorn and John is not available
Look Who's Cryin' Now by Jessie Murph is not available
Familiar Feeling by Moloko is not available
Settling Down by Miranda Lambert is not available
Working Class Man by Jimmy Barnes is not available
Missing You by John Waite is not available
Whiskey & Coca Cola by Isabella Lovestory is not available
The Last Goodbye by The Birthday Massacre is not available
Holiday b

The Creator Has A Master Plan by Brooklyn Funk Essentials is not available
Watch Over You by Alter Bridge is not available
Big Time by Louis The Child is not available
Bruises by Train is not available
How Do You Do It by Empress Of is not available
Never Surrender by Any Given Day is not available
Private Life by Allie X is not available
Boxes by The Goo Goo Dolls is not available
Loud and Heavy by Cody Jinks is not available
People Are People by Depeche Mode is not available
Discolights by Ultrabeat is not available
False Pretense by The Red Jumpsuit Apparatus is not available
Pedra Preta by Teto Preto is not available
Complicated by Poi Dog Pondering is not available
Automatic by ZHU is not available
LEILÃO by Gloria Groove is not available
Follow Me by Syn Cole is not available
The Last High by The Dandy Warhols is not available
Peter Gunn by Dave Grusin is not available
Elevate by Papa Roach is not available
RISE by League of Legends is not available
let go of your love by Thousan

Godspeed by James Blake is not available
Lightning In A Bottle by The Summer Set is not available
Bummerland by AJR is not available
White Wedding by Billy Idol is not available
Ignition by Matt Stillwell is not available
MULHER DO ANO XD by Luísa Sonza is not available
Innocence by Nathan Wagner is not available
Three Steps to Heaven by Showaddywaddy is not available
Television Sets by Alien Nosejob is not available
Premonition by Killstation is not available
Let's Dance by Chris Montez is not available
Mind Your Own Business by Delta 5 is not available
Hunting for Witches by Bloc Party is not available
HEART ATTACK by BRONSON is not available
Cum by Brooke Candy is not available
Midnight Rider by Train is not available
London Dungeon by Misfits is not available
Your Girlfriend by Blossoms is not available
Shadow Of The Moon by Strawberry Girls is not available
Werewolf by Figure is not available
Drive All Night by NEEDTOBREATHE is not available
I Mean It REMIX by G-Eazy is not availa

Dead Presidents by Teddy is not available
Change My Life by Ashes Remain is not available
Alien Crime Lord by The Voidz is not available
Somebody's Gonna Get Their Head Kicked in Tonight by The Rezillos is not available
Only One by Machinedrum is not available
Sonic Prayer by Earthless is not available
Kansas City by The Mowgli's is not available
My Head & My Heart by Ava Max is not available
Insubordination by Voodoo Glow Skulls is not available
Letting Go by Wild Nothing is not available
Can't Take My Eyes Off You by Joseph Vincent is not available
7 by Catfish and the Bottlemen is not available
I Can Do Better Than That by Sherie Rene Scott is not available
Gone Daddy Gone by Violent Femmes is not available
Luna In The Sky Forever by Cvlt Ov the Svn is not available
Kerosene by Miranda Lambert is not available
Shooting Stars by Rival Sons is not available
Learn to Fly by Foo Fighters is not available
Connected by Stereo MC's is not available
Flyktsoda by Ebba Grön is not available
P

### Clean and Preprocess Lyrics

Prepare lyrics for analysis by cleaning and normalizing them.

In [23]:
pd.set_option('mode.chained_assignment', None)

In [40]:
df_rock_lyrics2 = df_rock_lyrics.dropna(subset=['lyrics']) #rock/alternative

In [41]:
df_rock_cleaned = clean_lyrics(df_rock_lyrics2,'lyrics','words') #rock/alternative

/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_47566/3095339618.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r'\n',' ') #replace '\n' character with space
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_47566/3095339618.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r'\[[^\[\]]*]','') #remove brackets and inside text
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_47566/3095339618.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r"\'\w*",'').str.replace(r'[^\w\d\s]+','') #remove extra characters


In [43]:
df_rock_norm = normalize_lyrics(df_rock_cleaned,'words') #rock/alternative

In [18]:
#clean lyrics
df_rock_cleaned = clean_lyrics(df_rock_lyrics2,'lyrics','words') #rock/alternative

/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_53479/3095339618.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r'\n',' ') #replace '\n' character with space
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_53479/3095339618.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r'\[[^\[\]]*]','') #remove brackets and inside text
/var/folders/2v/qnfbs7f539d_d_vxqj6lwsdr0000gn/T/ipykernel_53479/3095339618.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[new_col] = df[new_col].str.replace(r"\'\w*",'').str.replace(r'[^\w\d\s]+','') #remove extra characters


In [19]:
#normalize lyrics
df_rock_norm = normalize_lyrics(df_rock_cleaned,'words') #rock/alternative

In [44]:
rock_segments = []
for lyrics in df_rock_norm['lyrics']:
    rock_segments.append(segmenting(lyrics))

In [45]:
df_rock_norm['segments'] = rock_segments

In [50]:
merged_rock_df = pd.merge(df_rock_norm ,df_rock, on = ['track','artist'] )

In [51]:
merged_rock_df.to_csv('../Data/df_rock_30k_lyrics.csv')

In [12]:
depends = []
with open('/Users/benjaminfenison/Desktop/requirements_all.txt', 'r') as f:
    for line in f.readlines():
        if line.replace('\n','') not in depends:
            depends.append(line.replace('\n',''))

In [13]:
len(depends)

415

In [14]:
textfile = open("/Users/benjaminfenison/Desktop/requirements.txt", "w")
for element in depends:
    textfile.write(element + "\n")
textfile.close()